# GitHub Actions with access to AWS


If you are completely new to GitHub actions watch this ~10 minute [video](https://www.youtube.com/watch?v=mFFXuXjVgkU) first.

I wanted for Actions to run command my EC2 instance. To do that, we first need to provide user AWS credentials to GitHub. We can do this by creating a repo settings/secrets and variables/ then create environment I named "AWS" and copy/pasted AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY to the secrets. Below you can see reference to these. The needs to have policy `AmazonSSMFullAccess` attached to it.

Then I use actions/checkout@v4 and aws-actions/configure-aws-credentials@v4 and `aws ssm send-command` to run a command on my EC2 instance.

The `aws ssm` does the following:
- git pull latest code
- kill currently running streamlit app
- start a new streamlit app
- log everything to a file

```yml
name: Execute SSM SendCommand

on: 
  push:
    branches:
      - main

jobs:
  ssm-command:
    runs-on: ubuntu-latest
    environment: AWS
    steps:
    - name: Checkout repository
      uses: actions/checkout@v4

    - name: Configure AWS credentials
      uses: aws-actions/configure-aws-credentials@v4
      with:
        aws-access-key-id: ${{ secrets.AWS_ACCESS_KEY_ID }}
        aws-secret-access-key: ${{ secrets.AWS_SECRET_ACCESS_KEY }}
        aws-region: us-west-1 

    - name: Send SSM Command
      run: |
        aws ssm send-command \
            --document-name "AWS-RunShellScript" \
            --targets "Key=instanceids,Values=i-08b8b6691ed2e1b6d" \
            --parameters commands="date >> /home/ubuntu/output.log && sudo -u ubuntu git -C /home/ubuntu/blog pull >> /home/ubuntu/output.log 2>&1 && sudo pkill -f streamlit >> /home/ubuntu/output.log 2>&1 && nohup /home/ubuntu/miniconda/bin/streamlit run /home/ubuntu/blog/nbs/projects/myGPT/myGPT.py --server.enableCORS false --server.enableXsrfProtection false >> /home/ubuntu/output.log 2>&1" \
            --region us-west-1
```

